In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from datetime import datetime, time
from prophet import Prophet
import glob
import matplotlib.pyplot as plt

from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
# obtain and sort required raw data
degree_days = 'ew_cdd'
path = "RawData"

def extract_date_time(filename):
    """
    extract the date and time from the filename
    :param filename:
    :return:
    """
    parts = filename.split('.')
    date = parts[1]
    time = parts[2]
    return date, time

def get_date(df, file):
    """get the date from the dataframe and the time from the filename and combine them into a datetime object
    :param df: dataframe containing the date
    :param file: filename containing the time
    :return: datetime object
    """
    # date_str = df[df.iloc[:, 2] == 1].iloc[0]['Date']
    date_str = str(file.split('.')[1])
    time_str = str(file.split('.')[2])
    # date = datetime.strptime(date_str, '%Y-%m-%d')
    date = datetime.strptime(date_str, '%Y%m%d')
    time_value = time(int(time_str), 0)
    combined_datetime = datetime.combine(date.date(), time_value)
    return combined_datetime

degree_days = degree_days
ecmwf_files = glob.glob(path + f'/ecmwf.*.[01][02].{degree_days}.csv')
ecmwf_sorted_files = sorted(ecmwf_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[3:]

ecmwf_ens_files = glob.glob(path + f'/ecmwf-eps.*.[01][02].{degree_days}.csv')
ecmwf_ens_sorted_files = sorted(ecmwf_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

gfs_ens_bc_files = glob.glob(path + f'/gfs-ens-bc.*.[01][02].{degree_days}.csv')
gfs_ens_bc_sorted_files = sorted(gfs_ens_bc_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]

cmc_ens_files = glob.glob(path + f'/cmc-ens.*.[01][02].{degree_days}.csv')
cmc_ens_sorted_files = sorted(cmc_ens_files, key=lambda x: (x.split('.')[1], x.split('.')[2]))[2:]
for _ in range(2):
    set1 = set((extract_date_time(filename) for filename in ecmwf_sorted_files))
    set2 = set((extract_date_time(filename) for filename in ecmwf_ens_sorted_files))

    ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in set2]
    ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                              extract_date_time(filename) in set1]
    cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if extract_date_time(filename) in set1]

    master_set = set((extract_date_time(filename) for filename in cmc_ens_sorted_files))
    gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                               extract_date_time(filename) in master_set]

    master_set = set((extract_date_time(filename) for filename in gfs_ens_bc_sorted_files))

    ecmwf_sorted_files = [filename for filename in ecmwf_sorted_files if extract_date_time(filename) in master_set]
    ecmwf_ens_sorted_files = [filename for filename in ecmwf_ens_sorted_files if
                              extract_date_time(filename) in master_set]
    gfs_ens_bc_sorted_files = [filename for filename in gfs_ens_bc_sorted_files if
                               extract_date_time(filename) in master_set]
    cmc_ens_sorted_files = [filename for filename in cmc_ens_sorted_files if
                            extract_date_time(filename) in master_set]

### process raw data into individual components for model

In [ ]:
# y
# euro ensemble - values to predict
ecmwf_ens_9_14 = pd.DataFrame(columns=['ens(9,14)'])

for i in range(1, len(ecmwf_ens_sorted_files)):
    ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
    ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
    prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
    prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

    date = get_date(ecmwf_ens_df, ecmwf_ens_sorted_files[i])
    prev_date = get_date(prev_ecmwf_ens_df, ecmwf_ens_sorted_files[i - 1])
    d2 = str(date)[:10]
    d1 = str(prev_date)[:10]

    if d2 == d1:
        offset = 1
    else:
        offset = 0

    cur = ecmwf_ens_df['Value'].iloc[8:14].sum()
    prev = prev_ecmwf_ens_df['Value'].iloc[(8+offset):(14+offset)].sum()
    change = cur - prev

    new_row = pd.DataFrame(change, columns=ecmwf_ens_9_14.columns, index=[date])
    ecmwf_ens_9_14 = pd.concat([ecmwf_ens_9_14, new_row])

In [ ]:
# X1
# euro ensemble day 8
ecmwf_ens_8 = pd.DataFrame(columns=['ens(8)'])

for i in range(1, len(ecmwf_ens_sorted_files)):
    ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
    ecmwf_ens_df = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
    prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
    prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

    date = get_date(ecmwf_ens_df, ecmwf_ens_sorted_files[i])
    prev_date = get_date(prev_ecmwf_ens_df, ecmwf_ens_sorted_files[i - 1])
    d2 = str(date)[:10]
    d1 = str(prev_date)[:10]

    if d2 == d1:
        offset = 1
    else:
        offset = 0

    cur = ecmwf_ens_df['Value'].iloc[7:8].sum()
    prev = prev_ecmwf_ens_df['Value'].iloc[(7+offset):(8+offset)].sum()
    change = cur - prev

    new_row = pd.DataFrame(change, columns=ecmwf_ens_8.columns, index=[date])
    ecmwf_ens_8 = pd.concat([ecmwf_ens_8, new_row])

In [ ]:
# X2
# ecmwf(9,10) - ecmwf_ens-1(9,10)

ecmwf_9_10 = pd.DataFrame(columns=['ecmwf(9,10)'])

for i in range(1, len(ecmwf_sorted_files)):
    ecmwf_df = pd.read_csv(ecmwf_sorted_files[i])
    ecmwf_df = ecmwf_df[ecmwf_df[ecmwf_df.columns[2]] >= 1]
    prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
    prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

    date = get_date(ecmwf_df, ecmwf_sorted_files[i])
    prev_date = get_date(prev_ecmwf_ens_df, ecmwf_sorted_files[i - 1])
    d2 = str(date)[:10]
    d1 = str(prev_date)[:10]

    if d2 == d1:
        offset = 1
    else:
        offset = 0

    cur = ecmwf_df['Value'].iloc[8:10].sum()
    prev = prev_ecmwf_ens_df['Value'].iloc[(8+offset):(10+offset)].sum()
    change = cur - prev

    new_row = pd.DataFrame(change, columns=ecmwf_9_10.columns, index=[date])
    ecmwf_9_10 = pd.concat([ecmwf_9_10, new_row])

In [ ]:
# X3
# gfs(11,14) - ecmwf_ens-1(11,14)

gfs_11_14 = pd.DataFrame(columns=['gfs(11,14)'])

for i in range(1, len(gfs_ens_bc_sorted_files)):
    gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
    gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]
    prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i-1])
    prev_ecmwf_ens_df = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

    date = get_date(gfs_df, gfs_ens_bc_sorted_files[i])
    prev_date = get_date(prev_ecmwf_ens_df, ecmwf_ens_sorted_files[i - 1])
    d2 = str(date)[:10]
    d1 = str(prev_date)[:10]

    if d2 == d1:
        offset = 1
    else:
        offset = 0

    cur = gfs_df['Value'].iloc[10:14].sum()
    prev = prev_ecmwf_ens_df['Value'].iloc[(10+offset):(14+offset)].sum()
    change = cur - prev

    new_row = pd.DataFrame(change, columns=gfs_11_14.columns, index=[date])
    gfs_11_14 = pd.concat([gfs_11_14, new_row])

In [ ]:
# X4
# cmc(9,14) - gfs(9,14)

cmc_9_14 = pd.DataFrame(columns=['cmc(9,14)'])

for i in range(1, len(cmc_ens_sorted_files)):
    cmc_df = pd.read_csv(cmc_ens_sorted_files[i])
    cmc_df = cmc_df[cmc_df[cmc_df.columns[2]] >= 1]
    gfs_df = pd.read_csv(gfs_ens_bc_sorted_files[i])
    gfs_df = gfs_df[gfs_df[gfs_df.columns[2]] >= 1]

    date = get_date(cmc_df, cmc_ens_sorted_files[i])

    cmc = cmc_df['Value'].iloc[8:14].sum()
    gfs = gfs_df['Value'].iloc[8:14].sum()
    change = cmc - gfs

    new_row = pd.DataFrame(change, columns=cmc_9_14.columns, index=[date])
    cmc_9_14 = pd.concat([cmc_9_14, new_row])

# testing

In [ ]:
i = 0

d1 = ecmwf_ens_sorted_files[i].split('.')[1]
d2 = ecmwf_ens_sorted_files[i - 1].split('.')[1]

ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i])
v1 = ecmwf_ens_df[ecmwf_ens_df[ecmwf_ens_df.columns[2]] >= 1]
prev_ecmwf_ens_df = pd.read_csv(ecmwf_ens_sorted_files[i - 1])
v2 = prev_ecmwf_ens_df[prev_ecmwf_ens_df[prev_ecmwf_ens_df.columns[2]] >= 1]

In [ ]:
print(ecmwf_ens_sorted_files[i])
print(ecmwf_ens_sorted_files[i - 1])

In [ ]:
if d1 == d2:
    offset = 1
else:
    offset = 0

In [ ]:
a = v1.iloc[7:8]

In [ ]:
b = v2.iloc[7+offset:8+offset]

In [ ]:
a['Value'].sum() - b['Value'].sum()

In [ ]:
display(a)

In [ ]:
display(b)

In [ ]:
display(v1)

In [ ]:
display(v2)

In [ ]:
display(ecmwf_ens_8.head())

# model

In [ ]:
master_df = pd.concat([ecmwf_ens_9_14, ecmwf_ens_8, ecmwf_9_10, gfs_11_14, cmc_9_14], axis=1)

In [ ]:
data = master_df.copy()

In [ ]:
#get only noon forecast
data['hour'] = data.index.hour
data = data[data['hour'] == 12]
data.drop('hour', axis=1, inplace=True)

In [ ]:
display(data.columns)

In [ ]:
if 'hdd' in degree_days:
    print('getting betas for hdd')
    b1 = 0.76947604258751
    b2 = 0.6555933110283885
    b3 = 0.16964083221789059
    b4 = 0.06680336217511197

else:
    print('getting betas for cdd')
    b1 = 1.7687808214521186
    b2 =0.42218026473687276
    b3 = 0.1128916050374201
    b4 = 0.08538981339364801

In [ ]:
(b1*0.003) + (b2*0.088) + (b3*0.037) + (b4*-0.040)

In [ ]:
features = ['ens(8)', 'ecmwf(9,10)', 'gfs(11,14)', 'cmc(9,14)']
X = data[features]

In [ ]:
X

In [ ]:
label = 'ens(9,14)'
y = data[label]

In [ ]:
y

In [ ]:
regression_preds = (b1 * X.iloc[:,0]) + (b2 * X.iloc[:,1]) + (b3 * X.iloc[:,2]) + (b4 * X.iloc[:,3])

In [ ]:
y[-225:]

In [ ]:
mse = mean_squared_error(y[-225:], regression_preds[-225:])
mse # mean squared error before any adjustments - benchmark to beat

In [ ]:
forecasting_errors = y - regression_preds

In [ ]:
plt.plot(forecasting_errors)

### forecasting errors linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, forecasting_errors, test_size=0.15, shuffle=False)

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_test, y_test)

In [ ]:
reg.coef_

In [ ]:
error_preds = reg.predict(X_test)
(((error_preds[-225:] + regression_preds[-225:]) - y[-225:]) ** 2).mean() # mean squared error after adjustments(lin reg)

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(forecasting_errors.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(forecasting_errors, lags=40, ax=ax2)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(forecasting_errors, trend='c', order=(1,1,1))
res = mod.fit(disp=False)
print(res.summary())

In [ ]:
start_date = '2022-10-04'
end_date = '2023-05-16'
predict = res.predict(start=start_date, end=end_date)

In [ ]:
predict

In [ ]:
(((predict + regression_preds[-225:]) - y[-225:])** 2).mean() # mean squared error after adjustments(sarimax)

### forecasting errors - prophet

In [ ]:
errors = forecasting_errors.to_frame()
errors['ds'] = errors.index
errors.rename(columns={0:'y'}, inplace=True)
errors.reset_index(drop=True, inplace=True)

In [ ]:
train_len = 0.85
train_errors = errors.iloc[:int(len(errors)*train_len)]
test_errors = errors.iloc[int(len(errors)*train_len):]

In [ ]:
m = Prophet()
m.fit(train_errors)

In [ ]:
future = m.make_future_dataframe(periods=263)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
forecasted_errors = forecast[['ds', 'yhat']]
forecasted_errors.set_index('ds', inplace=True)

In [ ]:
# errors without adjustment
((regression_preds[-225:] - y[-225:]) **2).mean()

In [ ]:
# errors with prophet adjustment
(((regression_preds[-225:] + forecasted_errors['yhat'][-225:]) - y[-225:])**2).mean()

### forecasting errors - autogluon

In [ ]:
data_df = data.copy()

In [ ]:
data_df['errors'] = forecasting_errors

In [ ]:
data_df.drop('ens(9,14)', axis=1, inplace=True)

In [ ]:
label = 'errors'
features = ['ens(8)', 'ecmwf(9,10)', 'gfs(11,14)', 'cmc(9,14)']

In [ ]:
train_len = 0.85

In [ ]:
train_data = TabularDataset(data_df.iloc[:int(len(data_df)*train_len)])
test_data = TabularDataset(data_df.iloc[int(len(data_df)*train_len):])

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
predictor = TabularPredictor(label=label).fit(train_data, presets='best_quality')

In [ ]:
test_data_no_lab = test_data.drop(columns=[label])
test_data_no_lab

In [ ]:
error_pred = predictor.predict(test_data_no_lab)
error_pred.head()

In [ ]:
predictor.evaluate(test_data, silent=True)

In [ ]:
# MSE after autogluon adjustment
(((error_pred[-225:] + regression_preds[-225:]) - y)**2).dropna().mean()

In [ ]:
error_pred[-225:]

In [ ]:
import xgboost as xgb

In [ ]:
label_df = train_data[label]
train_data = train_data[features]

In [ ]:
dtrain = xgb.DMatrix(train_data, label=label_df)

In [ ]:
bst = xgb.train(params={'objective': 'reg:squarederror'}, dtrain=dtrain)

In [ ]:
test_data = test_data[features]

In [ ]:
dtest = xgb.DMatrix(test_data)
ypred = bst.predict(dtest)

In [ ]:
# MSE after xgboost adjustment
(((ypred[-225:] + regression_preds[-225:]) - y[-225:]) ** 2).mean()

In [ ]:
xgb.plot_importance(bst)

### improving model through deep learning

In [ ]:
deep_preds = pd.read_pickle("predictions_cdd.pkl")

In [ ]:
deep_preds = deep_preds.to_frame()

In [ ]:
deep_preds.rename(columns={0:'prediction'}, inplace=True)

In [ ]:
y = y.to_frame()

In [ ]:
#combine deep_pred and y
deep_preds['y'] = y['ens(9,14)']

In [ ]:
deep_preds.dropna(inplace=True)

In [ ]:
# MSE of custom deep learning model
((deep_preds[deep_preds.columns[0]][-225:] - deep_preds[deep_preds.columns[1]][-225:])**2).mean()

In [ ]:
display(deep_preds[-225:])